In [1]:
import os
import time
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pylab as plt
%matplotlib inline
from sklearn.cluster import Birch
from sklearn import preprocessing

## Step 0: Data collection

In [2]:
print("Loading dataset...")

# path = 'training_data/2020_05_04/trace/'
path = 'sample/'
data = pd.read_csv(path + 'trace_data_sample.csv', index_col=False)
data.drop('Unnamed: 0', axis=1, inplace=True)
trace_data = data
print("--- Loaded trace data!")

path = 'training_data/2020_05_04/host/'
data = pd.read_csv(path + 'full_kpi_data.csv', index_col=False)
data.drop('Unnamed: 0', axis=1, inplace=True)
host_data = data
print("--- Loaded host data!")

del data, path
print("Data successfully loaded!\n")

Loading dataset...
--- Loaded trace data!
--- Loaded host data!
Data successfully loaded!



## Step 1: Data Pre-Processing (Done Manually)
- Concat all trace and host data files together
- Sort the rows based on Timestamp
- For trace data, if there's dsname, remove serviceName and replace it with dsname

## Step 2: Anomaly Detection (TODO)

In [37]:
# TODO! fix birch to find anomalies, currently it returns []

def birch_anomaly_detection(data, threshold):
    anomalies = []
    for svc, latency in data.iteritems():
        if svc == 'elapsedTime':
            latency = latency.rolling(window=1, min_periods=1).mean()
            x = np.array(latency)
            x = np.where(np.isnan(x), 0, x)
            normalized_x = preprocessing.normalize([x])

            X = normalized_x.reshape(-1,1)
            brc = Birch(branching_factor=10, compute_labels=True)
            brc.fit(X)
            brc.predict(X)

            labels = brc.labels
            n_clusters = np.unique(labels).size
            return anomalies

birch_anomaly_detection(data, 0)

[]

## Step 3: Creating the attributed graph (TODO)

In [ ]:
def attributed_graph(trace_data):
    DG = nx.DiGraph()
    services = trace_data.unique() # All the available service names

    for service in services:
        


attributed_graph(trace_data)

## Step 4: Anomalous subgraph extraction (TODO)

## Step 5: Anomaly localization (TODO) 

## Random testing

In [22]:
# Finding pid's are repeated once or twice for traces
def attributed_graph(trace_data):
        DG = nx.DiGraph()
        df = pd.DataFrame(columns=['source', 'destination'])
        print('started loop')

        serviceNames = trace_data.serviceName.dropna().unique()

        temp = {}
        for service in (serviceNames):
            # print(service)
            filtered = trace_data[trace_data.serviceName == service]
            temp[service]= {}
            for i in range(len(filtered)):
                trace = filtered.iloc[i] # Select the first one
                if trace['cmdb_id'] in temp[service].keys():
                    temp[service][trace['cmdb_id']] +=1
                else:
                    temp[service][trace['cmdb_id']] =1
            
        return temp
        # *Note* I tried different values of selected rows, no difference
        destination = trace['serviceName']
        assert destination == service

            # if trace['pid'] != 'None':
            #     _temp = True
            #     i = 0
            #     filtered = ''
            #     while _temp:
            #         trace = filtered.iloc[i]
            #         i += 1
            #         filtered = trace_data[trace_data['id'] == trace['pid']]
            #         _temp = filtered.empty
            #     parent = filtered.iloc[0]
            #     if parent['callType'] == 'JDBC':
            #         source = filtered.dsName.iloc[0]
            #     else:
            #         source = filtered.serviceName.iloc[0]
            #     # print('The source for service %s is %s' % (destination, source))
            #     df = df.append(
            #         {'source': source, 'destination': destination}, ignore_index=True)
            #     DG.add_edge(source, destination)
            #     DG.nodes[source]['type'] = 'service'

            # host = trace['cmdb_id']
            # # print('The host for service %s is %s' % (service, host))
            # DG.add_edge(destination, host)
            # DG.nodes[destination]['type'] = 'service'
            # DG.nodes[host]['type'] = 'host'
            # df = df.append(
            #     {'source': destination, 'destination': host}, ignore_index=True)
        
        # return DG

temp = attributed_graph(trace_data)

started loop


In [23]:
temp


{'osb_001': {'os_021': 4886, 'os_022': 4885},
 'csf_001': {'os_021': 4886,
  'docker_003': 2450,
  'docker_004': 2436,
  'os_022': 4885,
  'docker_001': 2437,
  'docker_002': 2448},
 'fly_remote_001': {'docker_004': 2436,
  'docker_001': 2437,
  'docker_002': 2448,
  'docker_003': 2450},
 'csf_002': {'docker_004': 2436,
  'docker_006': 4886,
  'docker_001': 2437,
  'docker_008': 2465,
  'docker_002': 2448,
  'docker_007': 2420,
  'docker_003': 2450},
 'db_003': {'docker_006': 78176, 'docker_008': 39141, 'docker_007': 39019},
 'csf_003': {'docker_004': 2436,
  'docker_006': 4886,
  'docker_002': 2448,
  'docker_008': 2419,
  'docker_007': 2466,
  'docker_001': 2437,
  'docker_003': 2450},
 'csf_004': {'docker_004': 2436,
  'docker_006': 4886,
  'docker_002': 2448,
  'docker_008': 2465,
  'docker_001': 2437,
  'docker_007': 2420,
  'docker_003': 2450},
 'csf_005': {'docker_004': 2436,
  'docker_006': 4886,
  'docker_002': 2448,
  'docker_007': 2443,
  'docker_008': 2442,
  'docker_001': 

In [9]:
DG.nodes

NodeView(('osb_001', 'os_021', 'csf_001', 'fly_remote_001', 'docker_004', 'csf_002', 'db_003', 'docker_006', 'csf_003', 'csf_004', 'csf_005', 'db_009', 'db_007'))

In [14]:
DG.edges

OutEdgeView([('osb_001', 'os_021'), ('osb_001', 'csf_001'), ('csf_001', 'os_021'), ('csf_001', 'fly_remote_001'), ('csf_001', 'csf_002'), ('csf_001', 'csf_003'), ('csf_001', 'csf_004'), ('csf_001', 'csf_005'), ('csf_001', 'db_009'), ('fly_remote_001', 'docker_004'), ('csf_002', 'docker_004'), ('db_003', 'db_003'), ('db_003', 'docker_006'), ('csf_003', 'docker_004'), ('csf_004', 'docker_004'), ('csf_005', 'docker_004'), ('db_009', 'docker_004'), ('db_007', 'db_007'), ('db_007', 'docker_004')])

In [15]:
DG.nodes

NodeView(('osb_001', 'os_021', 'csf_001', 'fly_remote_001', 'docker_004', 'csf_002', 'db_003', 'docker_006', 'csf_003', 'csf_004', 'csf_005', 'db_009', 'db_007'))